In [2]:
import requests 

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
import pandas as pd
import numpy as np

In [29]:
def extrae_datos_meneos (lista_meneos):
    datos = []
    continuar = True
    for meneo in lista_meneos:
        dicc_aux = {}
        
        # Se obtiene en primer lugar el año
        fecha_pub = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")[1].text.split(" ")[0]
        año = datetime.strptime(fecha_pub,"%d/%m/%Y").year if len(fecha_pub) > 5 else 2024
        
        # Si el año es el 2023, se continúa con la importación (para omitir las de 2024)
        if año == 2023:

            ###################################   Comunidad (Temática) ############################################################
            try:
                dicc_aux["Comunidad"] = meneo.find("a", class_="subname").text.strip()
            except:
                dicc_aux["Comunidad"] = np.nan

            ###################################    Usuario  ######################################################################
            try:
                dicc_aux["Usuario"] = meneo.find("div", class_="news-submitted").find("img")["alt"]
            except:
                dicc_aux["Usuario"] = np.nan

            ###################################    Medio    ######################################################################
            try:
                dicc_aux["Medio"] = meneo.find("span", class_="showmytitle").text.strip()
            except:            
                dicc_aux["Medio"] = "meneame.net"

            ###################################    Fechas    ######################################################################    
            try:
                fechas = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")
                dicc_aux["Enviado"] = fechas[0].text
                dicc_aux["Publicado"] = fechas[1].text
            except:
                dicc_aux["Enviado"] = np.nan
                dicc_aux["Publicado"] = np.nan

            ###################################    Titular    ####################################################################
            try:
                dicc_aux["Titular"] = meneo.find("h2").text.strip()
            except:
                dicc_aux["Titular"] = np.nan

            ###################################    Entradilla    ###################################################################
            try:
                dicc_aux["Entradilla"] = meneo.find("div", class_="news-content").text.strip()
            except:
                dicc_aux["Entradilla"] = np.nan
        
            ###################################    Votos    ######################################################################
            try:
                dicc_aux["Votos"] = meneo.find("div", class_="votes").find("a").text
            except:
                dicc_aux["Votos"] = np.nan

            ###################################    Click    ######################################################################
            try:
                dicc_aux["Clicks"] = meneo.find("div", class_="clics").find("span").text
            except:
                dicc_aux["Clicks"] = np.nan

            ###################################    Positivos    ###################################################################
            try:
                dicc_aux["Positivos"] = meneo.find("span", class_="positive-vote-number").text
            except:
                dicc_aux["Positivos"] = np.nan

            ###################################    Votos Anónimos    ##############################################################
            try:
                dicc_aux["Anonimos"] = meneo.find("span", class_="anonymous-vote-number").text
            except:
                dicc_aux["Anonimos"] = np.nan

            ###################################    Votos Negativos    ##############################################################
            try:
                dicc_aux["Negativos"] = meneo.find("span", class_="negative-vote-number").text
            except:
                dicc_aux["Negativos"] = np.nan

            ###################################    Votos Comentarios    #############################################################
            try:
                dicc_aux["Comentarios"] = meneo.find("a", class_="comments")["data-comments-number"]
            except:
                dicc_aux["Comentarios"] = np.nan

            ###################################    Karma    ########################################################################
            try:
                dicc_aux["Karma"] = meneo.find("span", class_="karma-number").text
            except:
                dicc_aux["Karma"] = np.nan

            datos.append(dicc_aux)

        # Si se ha alcanzado el año es 2022, se detendrá la importación
        elif año == 2022:
            continuar = False

    return [datos, continuar]

In [30]:
######################################## BLOQUE PARA PRUEBAS ##############################################################

# Para obtener todo el año 2023, empezar con la pagina 128 y eliminar la condición de "contador" en el bucle ####

######################################## BLOQUE PARA PRUEBAS ##############################################################

browser = webdriver.Chrome()
pagina = 728
lista_aux = []
contador = 0
continuar = True

while contador < 5 and continuar == True:
    try:
        browser.get(f"https://old.meneame.net/?page={pagina}")        
        Soup_pagina_meneos = BeautifulSoup(browser.page_source, "html.parser")
        lista_meneos = Soup_pagina_meneos.find_all("div", class_="news-summary")
        resultado = extrae_datos_meneos(lista_meneos)
        lista_aux.extend(resultado[0])
        continuar = resultado[1]
        pagina += 1
        contador += 1
    except:
        print(f"Error en página {pagina}")    

print(f"Total paginas importadas: {contador}")

browser.close()

Total paginas importadas: 3


In [31]:
df = pd.DataFrame(lista_aux)
df

,Comunidad,Usuario,Medio,Enviado,Publicado,Titular,Entradilla,Votos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,actualidad,MiguelDeUnamano,ctxt.es,02/01/2023 15:44,02/01/2023 20:30,Lula y la corrupción en el periodismo,Los informativos de TVE y Antena 3 destacaron ...,717,1131,300,417,4,41,475
1,cultura,anor,niusdiario.es,02/01/2023 15:35,02/01/2023 20:05,"Insomnio, la epidemia silenciosa que le roba e...",Siete millones de españoles sufren insomnio cr...,222,1687,100,122,0,129,455
2,actualidad,Beltenebros,eldiario.es,02/01/2023 15:47,02/01/2023 19:35,La Inspección de Trabajo investiga a la empres...,"""Es una conducta intolerable que atenta contra...",469,1334,199,270,4,95,461
3,ocio,Miguel.Lacambra,youtube.com,02/01/2023 17:20,02/01/2023 19:00,Ancianos chinos hacen 'ejercicios de cuello' c...,Un grupo de ancianos de la provincia de Liaoni...,107,2606,65,42,1,56,472
4,actualidad,Ratoncolorao,publico.es,02/01/2023 18:06,02/01/2023 18:35,El primer 'tropezón' del año de 'El programa d...,"Este lunes, primer día laborable del 2023, era...",681,6721,249,432,3,91,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,tecnología,--682766--,sincrogo.com,01/01/2023 00:34,01/01/2023 03:25,Cambios en la IT: Los trabajadores ya no tendr...,¡Importante!: Los trabajadores ya no tendrán q...,260,3963,123,137,1,57,509
62,ocio,Andaui,letraslibres.com,31/12/2022 21:05,01/01/2023 02:30,El gol imposible de Pelé,"Gracias a Pelé, un jugador alegre, juvenil y l...",82,5383,40,42,7,38,427
63,tecnología,ccguy,xatakamovil.com,01/01/2023 00:35,01/01/2023 01:25,"Por increíble que parezca, mi aplicación favor...",Si hay una app que me ha gustado y se ha conve...,310,9966,143,167,9,101,477
64,ciencia,robustiano,francis.naukas.com,31/12/2022 22:36,01/01/2023 00:55,Nuevo récord de factorización de números enter...,El algoritmo de Shor para factorizar números e...,142,1705,73,69,1,14,490
